# Sample Problem #3 with Gurobi

## Leveraging Optimization for Organizational Redesign Projects

![Org Redesign](orgRedesign.png)

In [1]:
import gurobipy as grb
import pandas as pd
import numpy as np

## Data Prep

#### Employees, Old department, Position, and Salary

In [2]:
EmployeeAttributes = pd.read_excel('EmployeeAttributes.xlsx').fillna(0)
EmployeeAttributes

,Name,Department,Salary,Marketing,Sales,Accounting
0,Michael,Old_DepA,75000,1.0,0.0,0.0
1,Gina,Old_DepA,100000,0.0,1.0,0.0
2,Matt,Old_DepA,95000,0.0,0.0,1.0
3,Brenda,Old_DepB,75000,1.0,0.0,0.0
4,Lydia,Old_DepB,93000,0.0,1.0,0.0
5,Chris,Old_DepB,130000,0.0,0.0,1.0
6,Katie,Old_DepC,85000,1.0,0.0,0.0
7,Gil,Old_DepC,65000,0.0,1.0,0.0
8,Erin,Old_DepC,58000,0.0,0.0,1.0


#### Employee Matrix (Who Works With Who)

In [3]:
EmployeeMatrix = pd.read_excel('WhoWorksWithWho.xlsx').fillna(0).astype(int)
EmployeeMatrix.replace(0,'')

,Michael,Gina,Matt,Brenda,Lydia,Chris,Katie,Gil,Erin
Michael,,1,1,,,,,,
Gina,1,,1,,,,,,
Matt,1,1,,,,,,,
Brenda,,,,,1,1,,,
Lydia,,,,1,,1,,,
Chris,,,,1,1,,,,
Katie,,,,,,,,1,1
Gil,,,,,,,1,,1
Erin,,,,,,,1,1,


#### New Org Structure and Requirements

In [4]:
NewOrgEmpty = pd.read_excel('NewOrgEmpty.xlsx')
NewOrgEmpty

,Department,Position,Number
0,New_DepA,Marketing,1
1,New_DepA,Sales,1
2,New_DepA,Accounting,1
3,New_DepB,Marketing,1
4,New_DepB,Sales,1
5,New_DepB,Accounting,1
6,New_DepC,Marketing,1
7,New_DepC,Sales,1
8,New_DepC,Accounting,1


#### New Department Budgets

In [5]:
budget = pd.read_excel('NewOrgBudgets.xlsx').set_index('Department').to_dict()['Budget']
budget

{'New_DepA': 250000, 'New_DepB': 273000, 'New_DepC': 280000}

#### Some other data prep stuff

In [6]:
employees = list(EmployeeAttributes.Name.unique())
salary = EmployeeAttributes[['Name','Salary']].set_index('Name').to_dict()['Salary']
departments = list(NewOrgEmpty.Department.unique())
positionsCountByDep = NewOrgEmpty.groupby(by=['Department','Position'],axis=0).sum().to_dict(orient='dict')['Number']
positions = list(NewOrgEmpty.Position.unique())
employeePosition = {}
for e in employees:
    for p in positions:
        employeePosition[e,p] = EmployeeAttributes.set_index('Name').loc[e,p]

#### Define the model

In [7]:
try:
    m.reset()
except:
    m = grb.Model('ReOrg')

## Decision Variables

#### Should employee e be assigned to department d?

In [8]:
x = m.addVars(employees,departments, vtype=grb.GRB.BINARY,name='x')

#### Do employees a and b work together?

In [9]:
y = m.addVars(employees,employees,departments,vtype=grb.GRB.BINARY,name='y')

## Objective

#### Maximize the occurences of people who used to work together still working together in the new organization


In [10]:
obj = m.setObjective(grb.quicksum(EmployeeMatrix.loc[a,b]*y[a,b,d] for a in employees for b in employees for d in departments),grb.GRB.MAXIMIZE)

## Constraints

#### Position requirements in the new org must be met


In [11]:
C1 = m.addConstrs(grb.quicksum(x[e,d]*employeePosition[e,p] for e in employees)==positionsCountByDep[d,p] for d in departments for p in positions)

#### Each department must abide by budget constraint


In [12]:
#C2 = m.addConstrs(grb.quicksum(x[e,d]*salary[e] for e in employees) <= budget[d] for d in departments)

#### Each employee must be placed in one and only one department


In [13]:
C3 = m.addConstrs((grb.quicksum(x[e,d] for d in departments)==1 for e in employees))

#### Two employees can only work together if they are assigned to the same department


In [14]:
for a in employees: 
    for b in employees:
        for d in departments:
            if a != b:
                m.addGenConstrAnd(y[a,b,d],[x[a,d],x[b,d]])

## Solution

In [15]:
m.optimize()

Optimize a model with 18 rows, 270 columns and 54 nonzeros
Model has 216 general constraints
Variable types: 0 continuous, 270 integer (270 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve added 108 rows and 0 columns
Presolve removed 0 rows and 189 columns
Presolve time: 0.00s
Presolved: 126 rows, 81 columns, 270 nonzeros
Variable types: 0 continuous, 81 integer (81 binary)
Found heuristic solution: objective 18.0000000

Root relaxation: cutoff, 78 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0        18.00000   18.00000  0.00%     -    0s

Explored 0 nodes (78 simplex iterations) in 0.16 seconds
Thread count was 4 (of 4 available processors)

Solution count 1: 18 

Optimal solution found (tolerance 1.0

In [16]:
print(round(m.getObjective().getValue()/2/len(employees)*100),'% of the employee matrix has been maintained')

100 % of the employee matrix has been maintained


In [17]:
EmployeeMatrix.replace(0,'')

,Michael,Gina,Matt,Brenda,Lydia,Chris,Katie,Gil,Erin
Michael,,1,1,,,,,,
Gina,1,,1,,,,,,
Matt,1,1,,,,,,,
Brenda,,,,,1,1,,,
Lydia,,,,1,,1,,,
Chris,,,,1,1,,,,
Katie,,,,,,,,1,1
Gil,,,,,,,1,,1
Erin,,,,,,,1,1,


In [18]:
for d in departments:
    print('')
    print(d)
    for a in employees:
        printed = 0
        for b in employees:
            
            if y[a,b,d].X == 1 and printed==0:
                print(' ',a)
                printed=1  


New_DepA
  Michael
  Gina
  Matt

New_DepB
  Brenda
  Lydia
  Chris

New_DepC
  Katie
  Gil
  Erin
